# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

# Load dataset

In [3]:
train_dir = '../input/major-data/MAJOR'

Name=[]
for file in os.listdir(train_dir):
    Name+=[file]
print(Name)
print(len(Name))

['virabhadrasana i', 'padmasana', 'savasana', 'vajrasana', 'vrischikasana', 'vriksasana', 'utthita hasta padangustasana', 'virabhadrasana ii', 'ustrasana', 'dandasana', 'bakasana', 'tittibhasana', 'tolasana', 'salamba sarvangasana', 'prasarita padottanasana', 'uttanasana', 'tadasana', 'parivrtta janu sirsasana', 'parivrtta trikonasana', 'paripurna navasana', 'lolasana', 'ardha pincha mayurasana', 'simhasana', 'natarajasana', 'hanumanasana', 'durvasasana']
26


In [4]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [7]:
def mapper(value):
    return reverse_mapping[value]

dataset=[]
testset=[]
count=0
for file in os.listdir(train_dir):
    t=0
    path=os.path.join(train_dir,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(40,40))
        image=img_to_array(image)
        image=image/255.0
        if t < 70:
            dataset+=[[image,count]]
        else:
            testset+=[[image,count]]
        t+=1
    count=count+1

data,labels0=zip(*dataset)
test,testlabels0=zip(*testset)

labels1=to_categorical(labels0)
labels=np.array(labels1)

data=np.array(data)
test=np.array(test)
    

# Taken very few images for  test 26 images

In [11]:
len(test)

26

# Split dataset

In [8]:
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

# Keras data pre processing

In [ ]:
datagen = ImageDataGenerator(
                                horizontal_flip=True,
                                vertical_flip=True,
                                rotation_range=20,
                                zoom_range=0.2,    
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.1,
                                fill_mode="nearest"
                            )

# Define model

In [ ]:
pretrained_model = tf.keras.applications.DenseNet201(
                                                        input_shape=(40,40,3),
                                                        include_top=False,
                                                        weights='imagenet',
                                                        pooling='avg'
                                                    )
pretrained_model.trainable = False

inputs3 = pretrained_model.input

x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
outputs3 = tf.keras.layers.Dense(26, activation='softmax')(x3)

model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=100)

# Predict 

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))


# Save Model

In [ ]:
model.save("Datta_Yoga_Pose.h5")

# Plot History

In [ ]:
get_acc = history.history['accuracy']
value_acc = history.history['val_accuracy']
get_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

# Test Model using sample image

In [ ]:
image=load_img("../input/major-data/MAJOR/lolasana/0-0.png",target_size=(40,40))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print("Prediction is {}.".format(move_name))

# Make prediction on test dataset

In [ ]:
print(test.shape)
pred2=model.predict(test)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]

ANS=testlabels0

accuracy=accuracy_score(ANS,PRED)
print(accuracy)